In [1]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import numpy as np

# --- 1. Leer los datos y preparar valores ---

try:
    # Asegúrate de que este archivo está en el mismo directorio
    spacex_df = pd.read_csv("spacex_launch_dash.csv")
except FileNotFoundError:
    print("ERROR: No se encontró 'spacex_launch_dash.csv'. Asegúrate de que el archivo existe.")
    # Crea un DataFrame vacío para evitar fallos si el archivo no existe
    spacex_df = pd.DataFrame(columns=['Launch Site', 'Payload Mass (kg)', 'class', 'Booster Version Category'])

# Definir los rangos de Payload. Usaremos valores por defecto si el DF está vacío.
if not spacex_df.empty:
    max_payload = spacex_df['Payload Mass (kg)'].max()
    min_payload = spacex_df['Payload Mass (kg)'].min()
else:
    # Valores de reserva si el CSV no carga
    max_payload = 10000
    min_payload = 0

# --- 2. Crear la aplicación Dash ---

app = dash.Dash(__name__)

# --- 3. Layout de la Aplicación ---

app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    # Dropdown de sitios
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'}
                 ] + [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()],
                 value='ALL',
                 placeholder="Select a Launch Site",
                 searchable=True,
                 style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'textAlignLast': 'center'}
    ),
    html.Br(),

    # Gráfico de torta (Pie Chart)
    html.Div(dcc.Graph(id='success-pie-chart'), style={'padding': '20px'}),
    html.Br(),

    html.P("Payload range (Kg):", style={'font-size': '20px', 'margin-top': '20px'}),

    # RANGO DE PAYLOAD (RANGE SLIDER) - TASK 3
    dcc.RangeSlider(
        id='payload-slider',
        min=0,         # Mínimo fijo del slider
        max=10000,     # Máximo fijo del slider
        step=500,      # Intervalo
        marks={i: {'label': str(i)} for i in range(0, 10001, 2000)}, # Marcas principales cada 2000
        value=[min_payload, max_payload], # Rango inicial basado en los datos cargados
        tooltip={"placement": "bottom", "always_visible": True} # Mostrar el valor seleccionado
    ),
    html.Br(),
    html.Br(),

    # Gráfico de dispersión (Scatter Chart)
    html.Div(dcc.Graph(id='success-payload-scatter-chart'))
])

# --- 4. Callbacks de la Aplicación ---

# CALLBACK TASK 2: pie chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if spacex_df.empty:
        # Retorna una figura vacía si no hay datos
        return px.pie(title="Error: Datos no cargados")

    if selected_site == 'ALL':
        # Cuenta la clase '1' (éxito) por sitio para el gráfico de torta
        success_counts = spacex_df[spacex_df['class'] == 1].groupby('Launch Site').size().reset_index(name='Successful Launches')
        fig = px.pie(success_counts, names='Launch Site', values='Successful Launches',
                     title='Total Successful Launches by Site',
                     hole=.3)
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        # Agrupa por 'class' (0 y 1) y usa size() para obtener los conteos
        class_counts = filtered_df.groupby('class').size().reset_index(name='Count')
        class_counts['class'] = class_counts['class'].map({1: 'Success (1)', 0: 'Failure (0)'})
        
        fig = px.pie(class_counts, names='class', values='Count',
                     title=f'Success vs Failure for site {selected_site}',
                     color='class',
                     color_discrete_map={'Success (1)':'#4CAF50', 'Failure (0)':'#F44336'})
    
    fig.update_layout(margin=dict(t=50, b=0, l=0, r=0))
    return fig

# CALLBACK TASK 4: scatter chart
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_chart(selected_site, payload_range):
    if spacex_df.empty:
        # Retorna una figura vacía si no hay datos
        return px.scatter(title="Error: Datos no cargados")
        
    low, high = payload_range
    
    # 1. Filtrar por rango de Payload
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & 
                            (spacex_df['Payload Mass (kg)'] <= high)]
    
    # 2. Filtrar por sitio si no es 'ALL'
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]

    # Crear el gráfico de dispersión
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                     color='Booster Version Category',
                     title=f'Correlation between Payload and Success for {selected_site}',
                     # Personalización del Scatter Plot
                     hover_data=['Launch Site'], # Mostrar info adicional al pasar el ratón
                     labels={'class': 'Launch Outcome (1=Success, 0=Failure)'},
                     )
                     
    # Asegurar que el eje Y solo muestre 0 y 1 claramente
    fig.update_layout(yaxis=dict(tickvals=[0, 1], ticktext=['Failure (0)', 'Success (1)'], range=[-0.1, 1.1]))
    
    return fig

# --- 5. Ejecutar app ---

if __name__ == '__main__':
    # Usar host='0.0.0.0' para que sea accesible externamente (si lo necesita)
    app.run(debug=True)